# Linkage disequlibrim

**authors:** Joseph Marcus, Hussein Al-Asadi

In our previous work we emphasized modeling genotypes being generated under the Wright-Fisher Markov Chain. This allowed us to model correlations across time and learn interesting population genetic parameters. Unfortunately, a lot of ancient DNA data is sparse through time and the structure of these correlations can actually be quite complicated (a single population model with constant effective size is unlikely). 


Maybe if we can shift our focus on obtaining the posterior distribution on the genotypes given the read data, taking advantage of the correlation structure due to LD. This approach has a advantage in that we observe many strongly correlated obersvations along the chromsome (in the reference panel) and the correlation structure depends on recombination. Of course there are cavets in this approach if there is population strucutre in the reference panel or our ancient individual is strongly diverged from the reference panel but it seems worthwhile to explore.

## Generative model

Lets consider a single individual where we observe read data $p$ SNPs. Following Wen and Stephens 2010 we let $\mathbf{g}_i$ be a $p$ vector of genotypes for the $i$th individual whose modeled as multivariate normal distribution. The first two moments ($\mu$,$\mathbf{\Sigma}$) of the distribution are analytically derived from the Li and Stephens model and the sample covariance matrix of the reference panel is shrunk to be banded and sparse, allowing increased computationally efficiency for inversion.  We extend this framework by adding an additional emission for the read data in which we consider a Normal approximation to the Binomial likelihood which allows us to have major boosts in computation efficiency.

$$
\begin{aligned}
\mathbf{g}_i | \sigma^2 &\sim \mathcal{N}\big(\hat{\mathbf{\mu}}, \sigma^2\hat{\mathbf{\Sigma}} \big) \\
\mathbf{y}_i | \mathbf{g}_i &\sim \mathcal{N}\Bigg(\mathbf{c}_i \cdot \frac{\mathbf{g}_i}{2}, diag\Big(\mathbf{c_i} \cdot \frac{\mathbf{g}_i}{2} \big(1 - \frac{\mathbf{g}_i}{2}\big)\Big) \Bigg)
\end{aligned}
$$

where $\mathbf{y}_i$ is a $p$ vector storing the number of reads that contain the counted allele, $\mathbf{c}_i$ is a $p$ vector storing the total number of reads. 

If we can somehow figure out a easy way to fix $diag\Big(\mathbf{c_i} \cdot \frac{\mathbf{g}_i}{2} \big(1 - \frac{\mathbf{g}_i}{2}\big)\Big)$ then we can take an Empirical Bayes approach obtaining the posterior distribution of the genotypes $\mathbf{g}_i | \mathbf{y}_i$ by integrating out $\mathbf{g}_i$ and using maximum likelihood to estimate $\sigma^2$. We can then plug in this point estimate into the model to obtain a conjugate update to compute the posterior distribution the genotypes given the read data. Note that this could be a highly parallelizable problem as we can compute posterior distributions in each individual separately and each chunk of a chromosome separately.